# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [110]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import hvplot.pandas
import bokeh

# Import API key
from api_keys import geoapify_key, gkey

In [111]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,skjervoy,70.0311,20.9714,1.00,81,100,4.83,NO,1667199635
1,1,khandyga,62.6667,135.6000,-15.85,91,84,1.02,RU,1667255443
2,2,carovigno,40.7080,17.6550,21.82,74,7,4.66,IT,1667193487
3,3,la rioja,42.2500,-2.5000,10.88,76,55,1.13,ES,1667198491
4,4,makakilo city,21.3469,-158.0858,30.18,58,20,8.75,US,1667234096


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [112]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    xlabel = "Longitude",
    ylabel = "Latitude",
    title = "City Data",
    legend = False,
    alpha=0.5,
    size = "Humidity",
    color="City"
    )

# Display the map plot
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [113]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = (city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] < 24)
ideal_wind = city_data_df["Wind Speed"] < 5
ideal_cloud = city_data_df["Cloudiness"] <= 10

city_data_df = city_data_df[ideal_temp & ideal_wind & ideal_cloud]

city_data_df.head()

# Drop any rows with null values
city_data_df.dropna(how='any', inplace=True)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,carovigno,40.7080,17.6550,21.82,74,7,4.66,IT,1667193487
5,5,acari,-15.4311,-74.6158,20.32,68,6,2.33,PE,1667211748
22,22,caldwell,43.6629,-116.6874,21.16,37,0,1.34,US,1667226077
75,75,bilma,18.6853,12.9164,22.45,21,0,3.23,NE,1667192862
83,83,aswan,24.0934,32.9070,22.61,33,0,2.57,EG,1667274867
134,134,ahipara,-35.1667,173.1667,20.03,63,8,0.88,NZ,1667237121
169,169,grand gaube,-20.0064,57.6608,22.43,76,10,2.68,MU,1667266069
196,196,christchurch,-43.5333,172.6333,21.85,52,5,3.09,NZ,1667236335
224,224,namibe,-15.1961,12.1522,22.04,66,1,4.02,AO,1667190945
230,230,port said,31.2565,32.2841,22.99,64,0,2.57,EG,1667275568


### Step 3: Create a new DataFrame called `hotel_df`.

In [114]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,carovigno,IT,40.7080,17.6550,74,
5,acari,PE,-15.4311,-74.6158,68,
22,caldwell,US,43.6629,-116.6874,37,
75,bilma,NE,18.6853,12.9164,21,
83,aswan,EG,24.0934,32.9070,33,
134,ahipara,NZ,-35.1667,173.1667,63,
169,grand gaube,MU,-20.0064,57.6608,76,
196,christchurch,NZ,-43.5333,172.6333,52,
224,namibe,AO,-15.1961,12.1522,66,
230,port said,EG,31.2565,32.2841,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [115]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carovigno - nearest hotel: Borgo Antico
acari - nearest hotel: No hotel found
caldwell - nearest hotel: La Quinta Inn by Wyndham Caldwell
bilma - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
ahipara - nearest hotel: YHA Ahipara
grand gaube - nearest hotel: Coco Villa
christchurch - nearest hotel: Ibis Hotel Christchurch
namibe - nearest hotel: Hotel Chik Chik Namibe
port said - nearest hotel: The New Continental Hotel
san juan bautista - nearest hotel: Hacienda San Juan Resort & Spa
cocachacra - nearest hotel: No hotel found
bari - nearest hotel: Villa Romanazzi-Carducci
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
north myrtle beach - nearest hotel: Mar Vista Grande
marzuq - nearest hotel: No hotel found
lincoln - nearest hotel: New Victorian Suites Lincoln
dowa - nearest hotel: No hotel found
lander - nearest hotel: Rodeway Inn Pronghorn Lodge
naples - nearest hotel: Hotel Miramare
gaya - nearest hotel: Royal Gue

,City,Country,Lat,Lng,Humidity,Hotel Name
2,carovigno,IT,40.7080,17.6550,74,Borgo Antico
5,acari,PE,-15.4311,-74.6158,68,No hotel found
22,caldwell,US,43.6629,-116.6874,37,La Quinta Inn by Wyndham Caldwell
75,bilma,NE,18.6853,12.9164,21,No hotel found
83,aswan,EG,24.0934,32.9070,33,Yaseen hotel
134,ahipara,NZ,-35.1667,173.1667,63,YHA Ahipara
169,grand gaube,MU,-20.0064,57.6608,76,Coco Villa
196,christchurch,NZ,-43.5333,172.6333,52,Ibis Hotel Christchurch
224,namibe,AO,-15.1961,12.1522,66,Hotel Chik Chik Namibe
230,port said,EG,31.2565,32.2841,64,The New Continental Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [116]:
%%capture --no-display

hotel_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    xlabel = "Longitude",
    ylabel = "Latitude",
    title = "Hotel Map",
    legend = False,
    alpha=0.5,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
    )

    
# Display the map
hotel_data_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)